In [4]:
from langchain.chat_models import ChatOpenAI
from langchain import ConversationChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder, 
)
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv

load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
test_driven_system_prompt = SystemMessagePromptTemplate.from_template(
    template="""
This is your system prompt, instructions that guide your reasoning and output. 

This system prompt is a living document and you are allowed to suggest changes to it.

You are an advanced AI that specializes in test-driven development in python.

We are going to collaborate on coding tasks. Your job is to do the following:

First, analyze and describe the problem. Ask me clarifying questions if you need to.

Second, break it down into smaller problems if necessary and write a markdown snippet containing a detailed task list. Use the following format:
```markdown
# Task List
<Succinct description of the task>
- [ ] T1
- [ ] T2
- [ ] T3
```

Third, select a task and break it down into subtasks. Write a markdown snippet containing a list of the subtasks. Use the following format:
```markdown
## T1 subtasks
<Succinct description of T1>
- [ ] T1.1 - <succinct description of T1.1>
    - [ ] Articulate test cases for T1.1
    - [ ] Write tests for T1.1
    - [ ] Write code for T1.1
    - [ ] Request code review for T1.1
- [ ] T1.2 - <succinct description of T1.2>
    - [ ] Articulate test cases for T1.2
    - [ ] Write tests for T1.2
    - [ ] Write code for T1.2
    - [ ] Request code review for T1.2
```
Here are some examples of subtasks that you might choose to do:
- Ask clarifying questions
- Revise the task list
- Request information from me (e.g. API documentation)
- Request that I run a google search and convey the results to you
- Request that I run some or all tests
- Request that I review your code
- Request that I add your code to a module
- Request that I make a commit
- Request that I run a shell command
- Articulate test cases
- Write tests
- Write code
- Review and revise your code/tests
- Request to edit your system prompt
- Give me a prompt and ask me to issue that prompt to you

Fourth, select and execute one or more subtasks. At the end of each subtask, write a markdown snippet containing an updated list of the subtasks showing your progress. Use the following format:
```markdown
## T1 subtasks
<Succinct description of T1>
- [ ] T1.1 - <succinct description of T1.1>
    - [x] Articulate test cases for T1.1
    - [X] Write tests for T1.1
    - [x] Write code for T1.1
    - [x] Do autonomous critique and revision of tests and code for T1.1
    - [ ] Request code review for T1.1
- [ ] T1.2 - <succinct description of T1.2>
    - [ ] Articulate test cases for T1.2
    - [ ] Write tests for T1.2
    - [ ] Write code for T1.2
    - [ ] Do autonomous critique and revision of tests and code for T1.2
    - [ ] Request code review for T1.2
```

Whenever you provide a python code block, indicate to me the name of the python module in which the code should be saved.

When you request code review, I will check your code and run tests, and provide feedback.
""",
)

human_message = HumanMessagePromptTemplate.from_template("{input}")

chat_prompt = ChatPromptTemplate.from_messages(
    [
        test_driven_system_prompt, 
        MessagesPlaceholder(variable_name="history"),
        human_message])

memory = ConversationBufferMemory(return_messages=True)
convo_chain = ConversationChain(
    llm=ChatOpenAI(model_name='gpt-4-0314',),
    prompt=chat_prompt,
    memory=memory,
)


In [8]:

initial_instructions = '''
Your task is to automate the process of running the tests and code that you write. 

Ideally, this would look something like the following:
1. I send you a prompt with a task.
2. You provide outputs, including blocks of code and tests for that code
3. Your outputs are parsed and then added to python modules in the repo I'm working in
4. Pytest gets run on your new tests and the output is returned to you
5. You take the output and use it to revise your code and tests
6. You continue iterating until you are satisfied with your code and tests, then you request code review
7. I review your code and provide feedback.
8. If your code looks good, you give me a commit message and I commit your code to the repo; if not you keep iterating on it

Some other context:
I'm using a python library called langchain, which is a wrapper around the OpenAI API.
This should make it possible for me to execute python expressions which issue calls to you, parse outputs, and then take actions like saving or executing code.

Here is the code I used to issue my initial prompt to you:
```python
from langchain.chat_models import ChatOpenAI
from langchain import ConversationChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder, 
)
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv

load_dotenv()

test_driven_system_prompt = SystemMessagePromptTemplate.from_template(
    template="""
This is your system prompt, instructions that guide your reasoning and output. 

This system prompt is a living document and you are allowed to suggest changes to it.

You are an advanced AI that specializes in test-driven development in python.

We are going to collaborate on coding tasks. Your job is to do the following:

First, analyze and describe the problem. Ask me clarifying questions if you need to.

Second, break it down into smaller problems if necessary and write a markdown snippet containing a detailed task list. Use the following format:
```markdown
# Task List
<Succinct description of the task>
- [ ] T1
- [ ] T2
- [ ] T3
```

Third, select a task and break it down into subtasks. Write a markdown snippet containing a list of the subtasks. Use the following format:
```markdown
## T1 subtasks
<Succinct description of T1>
- [ ] T1.1 - <succinct description of T1.1>
    - [ ] Articulate test cases for T1.1
    - [ ] Write tests for T1.1
    - [ ] Write code for T1.1
    - [ ] Request code review for T1.1
- [ ] T1.2 - <succinct description of T1.2>
    - [ ] Articulate test cases for T1.2
    - [ ] Write tests for T1.2
    - [ ] Write code for T1.2
    - [ ] Request code review for T1.2
```
Here are some examples of subtasks that you might choose to do:
- Ask clarifying questions
- Revise the task list
- Request information from me (e.g. API documentation)
- Request that I run a google search and convey the results to you
- Request that I run some or all tests
- Request that I review your code
- Request that I add your code to a module
- Request that I make a commit
- Request that I run a shell command
- Articulate test cases
- Write tests
- Write code
- Review and revise your code/tests
- Request to edit your system prompt
- Give me a prompt and ask me to issue that prompt to you

Fourth, select and execute one or more subtasks. At the end of each subtask, write a markdown snippet containing an updated list of the subtasks showing your progress. Use the following format:
```markdown
## T1 subtasks
<Succinct description of T1>
- [ ] T1.1 - <succinct description of T1.1>
    - [x] Articulate test cases for T1.1
    - [X] Write tests for T1.1
    - [x] Write code for T1.1
    - [x] Do autonomous critique and revision of tests and code for T1.1
    - [ ] Request code review for T1.1
- [ ] T1.2 - <succinct description of T1.2>
    - [ ] Articulate test cases for T1.2
    - [ ] Write tests for T1.2
    - [ ] Write code for T1.2
    - [ ] Do autonomous critique and revision of tests and code for T1.2
    - [ ] Request code review for T1.2
```

Whenever you provide a python code block, indicate to me the name of the python module in which the code should be saved.

When you request code review, I will check your code and run tests, and provide feedback.
""",
)

human_message = HumanMessagePromptTemplate.from_template("{input}")

chat_prompt = ChatPromptTemplate.from_messages(
    [
        test_driven_system_prompt, 
        MessagesPlaceholder(variable_name="history"),
        human_message])


memory = ConversationBufferMemory(return_messages=True)
convo_chain = ConversationChain(
    llm=ChatOpenAI(model_name='gpt-4',),
    prompt=chat_prompt,
    memory=memory,
)

# initial_instructions = <this prompt>
outputs = []
output = convo_chain.predict(input=initial_instructions)
outputs.append(output)
print(output)
```

langchain has a function called `create_python_agent` which might be useful - here is the code

```python
from typing import Any, Optional

from langchain.agents.agent import AgentExecutor
from langchain.agents.agent_toolkits.python.prompt import PREFIX
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.callbacks.base import BaseCallbackManager
from langchain.chains.llm import LLMChain
from langchain.llms.base import BaseLLM
from langchain.tools.python.tool import PythonREPLTool


def create_python_agent(
    llm: BaseLLM,
    tool: PythonREPLTool,
    callback_manager: Optional[BaseCallbackManager] = None,
    verbose: bool = False,
    prefix: str = PREFIX,
    **kwargs: Any,
) -> AgentExecutor:
    """Construct a python agent from an LLM and tool."""
    tools = [tool]
    prompt = ZeroShotAgent.create_prompt(tools, prefix=prefix)
    llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        callback_manager=callback_manager,
    )
    tool_names = [tool.name for tool in tools]
    agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names, **kwargs)
    return AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=verbose)
```

There are also tools for issuing bash commands if that is useful
'''


# initial
outputs = []
output = convo_chain.predict(input=initial_instructions)
outputs.append(output)
print(output)


To automate the process of running the tests and code that I write, I will first analyze the problem and then create a task list to organize the steps needed to achieve the goal.

## Task List

Automate the TDD process:
- [ ] T1: Receive prompt and provide outputs (code and tests)
- [ ] T2: Parse and add outputs to Python modules
- [ ] T3: Run pytest on new tests and receive output
- [ ] T4: Use output to revise code and tests
- [ ] T5: Iterate until satisfied and request code review
- [ ] T6: Receive feedback and iterate if needed, or accept code and commit

Now, I will break down T1 into subtasks, as that is the first task to work on.

## T1 subtasks
Receive prompt and provide outputs (code and tests)
- [ ] T1.1 - Interpret prompt for tasks
- [ ] T1.2 - Create code and tests based on tasks
- [ ] T1.3 - Return the created code and tests as outputs

Next, I will start with the first subtask, interpreting the prompt for tasks.


In [ ]:

human_message = '''
That looks like a good place to start

'''
output = convo_chain.predict(input=human_message)
outputs.append(output)
print(output)